In [2]:
import sys
print(sys.executable)

import torch
print("Torch:", torch.__version__)

import whisper
print("Whisper loaded successfully")

c:\Users\ishir\OneDrive\Desktop\Hackathons\AccessAIthon2026\venv\Scripts\python.exe
Torch: 2.10.0+cpu
Whisper loaded successfully


In [3]:
model = whisper.load_model("base")
result = model.transcribe("test_audio.wav")
print(result["text"])

c:\Users\ishir\OneDrive\Desktop\Hackathons\AccessAIthon2026\venv\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 This is a waste protest.
